# CPN generator

In [6]:
import pandas as pd
import numpy as np
import spacy
import gensim

In [7]:
# Definitivamente mostrar al profe y ver que se hace con esto
print(nlp.Defaults.stop_words)

{'cosas', 'pronto', 'sean', 'aun', 'empleas', 'tiempo', 'llegó', 'trabajar', 'queremos', 'verdad', 'habrá', 'aquél', 'pero', 'cuál', 'más', 'primeros', 'trabajo', 'veces', 'trabajais', 'sola', 'hablan', 'mio', 'dan', 'excepto', 'ella', 'menos', 'podrias', 'durante', 'manera', 'cuanta', 'sigue', 'mías', 'dice', 'debido', 'informó', 'señaló', 'ultimo', 'nuestros', 'sería', 'intentar', 'deben', 'se', 'entre', 'considera', 'esa', 'estoy', 'mia', 'mía', 'cuantos', 'esta', 'haciendo', 'vaya', 'ninguno', 'tercera', 'cuatro', 'además', 'soyos', 'ambos', 'tambien', 'ellas', 'llevar', 'propias', 'despacio', 'bastante', 'podriais', 'muy', 'quien', 'empleo', 'pueda', 'conseguimos', 'podriamos', 'podrán', 'le', 'saben', 'sobre', 'poner', 'aún', 'usan', 'parte', 'tendrán', 'tengo', 'sera', 'da', 'eras', 'van', 'nueva', 'ademas', 'expresó', 'míos', 'seis', 'hicieron', 'sabe', 'eramos', 'hubo', 'lugar', 'propio', 'usar', 'nuestra', 'dar', 'que', 'apenas', 'repente', 'vuestros', 'encuentra', 'fuera', '

## Funciones Importantes

In [34]:
def Normalize(text):
    """
    Recive a string and return the same string without punctions and 
    stop words
    """
     # Create a object spacy type nlp
    doc = nlp(text)

    # Remove all special caracters 
    words = [t.orth_ for t in doc if not (t.is_punct or t.is_stop)]
    tokens = [t.lower() for t in words]

    return(" ".join(tokens))

def Lematize(text):
    """
    Recive a string and return the string Lematized
    """
    doc = nlp(text)

    # Remove all special caracters 
    lemas = [t.lemma_ for t in tokens]

    return(" ".join(lemas))

def to_vector(texto,model):
    """ 
    Receives a sentence string along with a word embedding model and 
    returns the vector representation of the sentence
    """
    tokens = texto.split() # splits the text by space and returns a list of words
    vec = np.zeros(300) # creates an empty vector of 300 dimensions
    for word in tokens: # iterates over the sentence
        if word in model: # checks if the word is both in the word embedding
            vec += model[word] # adds every word embedding to the vector
    return vec / np.linalg.norm(vec) # divides the vector by their normal

In [32]:
Normalize("Hola a todos los últimos!!")

'hola a'

In [35]:
pd.options.mode.chained_assignment = None # Para evitar que se muestre warnings
# Normalizacion de los conceptos, descripciones y codificaciones
for i in range(len(data)):
    data['Palabra (concepto)'][i] = Normalize(data['Palabra (concepto)'][i])
    data['Descripción'][i] = Normalize(data['Descripción'][i]) 
    data['Codificación'][i] = Normalize(data['Codificación'][i])

## Pre-process

In [ ]:
# ? Puede ser el preprocesamiento algo importante que mostrar en mi tesis?
# ## Analysis Frequency Distribution of Word Counts in Documents

# doc_lens = [len(d) for d in df['sentence_clean']]

# #plot
# plt.figure(figsize=(16,7), dpi=160)
# plt.hist(doc_lens, bins = 200, color='navy')
# plt.text(150, 230, "Mean   : " + str(round(np.mean(doc_lens))))
# plt.text(150,  200, "Median : " + str(round(np.median(doc_lens))))
# plt.text(150,  170, "Stdev   : " + str(round(np.std(doc_lens))))
# plt.text(150,  140, "1%ile    : " + str(round(np.quantile(doc_lens, q=0.01))))
# plt.text(150,  110, "99%ile  : " + str(round(np.quantile(doc_lens, q=0.99))))

# plt.gca().set(xlim=(0, 200), ylabel='Number of Documents', xlabel='Document Word Count')
# plt.tick_params(size=16)
# plt.xticks(np.linspace(0,200,9))
# plt.title('Distribution of Document Word Counts', fontdict=dict(size=22))
# plt.show()

## Generación de Matriz de los datos

In [ ]:
# Creation of the matrix to make the clustering process
descriptions_matrix = np.zeros( # creatign an empty matrix
    (
        len(data), # the number of data points
        len(model['hola']) # the number of components of the word embedding
    )
)
# matrix filling 
for i,description in enumerate(data['Descripción']):
    vector = to_vector(description,model)
    descriptions_matrix[i,] = vector

# Concatenate the matrix with the data of each observation
data_matrix = np.concatenate([descriptions_matrix,data], axis=1)

#--------------------------------------------------------------------------------------
# This is the important matrix
# Remove of the 'Nan' data
data_matrix_without_nan = data_matrix[~pd.isnull(data_matrix[:,:300]).any(axis=1)]
#--------------------------------------------------------------------------------------

# reduce the matrix to the importan
data_matrix_without_nan = np.concatenate(
    (data_matrix_without_nan[:,:300],                       # Vector of the description
    np.reshape(data_matrix_without_nan[:,302], (-1, 1)),    # Cue/Concept
    np.reshape(data_matrix_without_nan[:,306], (-1, 1)),    # Codification
    np.reshape(data_matrix_without_nan[:,303], (-1, 1))     # Description of the cue
    ),
    axis = 1
)

In [ ]:
# data import
data = pd.read_excel(r'../data/Datos Codificados Estudio v2a.xlsx')
model = gensim.models.KeyedVectors.load_word2vec_format( 
    r'../Data/SBW-vectors-300-min5.bin', # using the spanish billion words embeddings
    binary=True # the model is in binary format
)

In [8]:
# Configuracion de spacy
nlp = spacy.load('es_core_news_sm')
# nlp.Defaults.stop_words.add("my_new_stopword")

In [4]:
# Definitivamente mostrar al profe y ver que se hace con esto
print(nlp.Defaults.stop_words)

{'cosas', 'pronto', 'sean', 'aun', 'empleas', 'tiempo', 'llegó', 'trabajar', 'queremos', 'verdad', 'habrá', 'aquél', 'pero', 'cuál', 'más', 'primeros', 'trabajo', 'veces', 'trabajais', 'sola', 'hablan', 'mio', 'dan', 'excepto', 'ella', 'menos', 'podrias', 'durante', 'manera', 'cuanta', 'sigue', 'mías', 'dice', 'debido', 'informó', 'señaló', 'ultimo', 'nuestros', 'sería', 'intentar', 'deben', 'se', 'entre', 'considera', 'esa', 'estoy', 'mia', 'mía', 'cuantos', 'esta', 'haciendo', 'vaya', 'ninguno', 'tercera', 'cuatro', 'además', 'soyos', 'ambos', 'tambien', 'ellas', 'llevar', 'propias', 'despacio', 'bastante', 'podriais', 'muy', 'quien', 'empleo', 'pueda', 'conseguimos', 'podriamos', 'podrán', 'le', 'saben', 'sobre', 'poner', 'aún', 'usan', 'parte', 'tendrán', 'tengo', 'sera', 'da', 'eras', 'van', 'nueva', 'ademas', 'expresó', 'míos', 'seis', 'hicieron', 'sabe', 'eramos', 'hubo', 'lugar', 'propio', 'usar', 'nuestra', 'dar', 'que', 'apenas', 'repente', 'vuestros', 'encuentra', 'fuera', '

In [34]:
def Normalize(text):
    """
    Recive a string and return the same string without punctions and 
    stop words
    """
     # Create a object spacy type nlp
    doc = nlp(text)

    # Remove all special caracters 
    words = [t.orth_ for t in doc if not (t.is_punct or t.is_stop)]
    tokens = [t.lower() for t in words]

    return(" ".join(tokens))

def Lematize(text):
    """
    Recive a string and return the string Lematized
    """
    doc = nlp(text)

    # Remove all special caracters 
    lemas = [t.lemma_ for t in tokens]

    return(" ".join(lemas))

def to_vector(texto,model):
    """ 
    Receives a sentence string along with a word embedding model and 
    returns the vector representation of the sentence
    """
    tokens = texto.split() # splits the text by space and returns a list of words
    vec = np.zeros(300) # creates an empty vector of 300 dimensions
    for word in tokens: # iterates over the sentence
        if word in model: # checks if the word is both in the word embedding
            vec += model[word] # adds every word embedding to the vector
    return vec / np.linalg.norm(vec) # divides the vector by their normal

In [32]:
Normalize("Hola a todos los últimos!!")

'hola a'

In [35]:
pd.options.mode.chained_assignment = None # Para evitar que se muestre warnings
# Normalizacion de los conceptos, descripciones y codificaciones
for i in range(len(data)):
    data['Palabra (concepto)'][i] = Normalize(data['Palabra (concepto)'][i])
    data['Descripción'][i] = Normalize(data['Descripción'][i]) 
    data['Codificación'][i] = Normalize(data['Codificación'][i])

In [ ]:
# Creation of the matrix to make the clustering process
descriptions_matrix = np.zeros( # creatign an empty matrix
    (
        len(data), # the number of rows is equal to the number of data points
        len(model['hola']) # the number of columns is equal to the number of components of the word embedding
    )
)
# matrix filling 
for i,description in enumerate(data['Descripción']):
    vector = to_vector(description,model)
    descriptions_matrix[i,] = vector

# Concatenate the matrix with the data of each observation
data_matrix = np.concatenate([descriptions_matrix,data], axis=1)

#--------------------------------------------------------------------------------------
# This is the important matrix
# Remove of the 'Nan' data
data_matrix_without_nan = data_matrix[~pd.isnull(data_matrix[:,:300]).any(axis=1)]
#--------------------------------------------------------------------------------------

# reduce the matrix to the importan
data_matrix_without_nan = np.concatenate(
    (data_matrix_without_nan[:,:300],                       # Vector of the description
    np.reshape(data_matrix_without_nan[:,302], (-1, 1)),    # Cue/Concept
    np.reshape(data_matrix_without_nan[:,306], (-1, 1)),    # Codification
    np.reshape(data_matrix_without_nan[:,303], (-1, 1))     # Description of the cue
    ),
    axis = 1
)